### 0. Load environment variables

In [1]:
import os, sys
from dotenv import load_dotenv
_ = load_dotenv('.env')
sys.path.extend(['../..', '..'])

### 1. Get an example of input variables for the prompt

In [14]:
from utils.pydantic_models.agent_trajectory import Trajectory
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
trajectory_getting_func = convert_pydantic_to_openai_function(Trajectory)


task = "Which is bigger? BTS's oldest member's birth month vs. incumbent Korean president's birth month"
scratchpad = """Entering new AgentExecutor chain...
Thought: I need to find out the birth month of the oldest member of BTS and the birth month of the current Korean president. 

Action:
{
  "action": "GetFromYDC",
  "action_input": {
    "Query": "Oldest member of BTS birth month"
  }
}

The oldest member of BTS is Jin, and his birth month is December. He was born on December 4, 1992.Action:

Action:

{
  "action": "GetFromYDC",
  "action_input": {
    "Query": "Current Korean president birth month"
  }
}

The current President of South Korea, Yoon Suk Yeol, was born on 18 December 1960. So, his birth month is December.

Action:
{
  "action": "Final Answer",
  "action_input": "The birth month of the oldest member of BTS, Jin, and the birth month of the current Korean president, Yoon Suk Yeol, are both December. Therefore, neither is bigger as they are the same."
}


Finished chain."""

from typing import Literal
from langchain import hub
from langchain.chat_models import ChatOpenAI, AzureChatOpenAI
from utils.custom_output_parsers.functioncall_output_parsers import ReActOutputJsonKeyStrParser


def get_OpenAI_chain_1(provider: Literal['ChatOpenAI', 'AzureChatOpenAI'], model_or_deployment_name: str):
    if provider == 'ChatOpenAI':
        chat_model = ChatOpenAI(model_name=model_or_deployment_name, temperature=0).bind(functions=[trajectory_getting_func], function_call={'name':'Trajectory'})
    else:
        chat_model = AzureChatOpenAI(deployment_name=model_or_deployment_name, temperature=0).bind(functions=[trajectory_getting_func], function_call={'name':'Trajectory'})
    return chat_model

prompt_1 = hub.pull("jet-taekyo-lee/tagging-extracting-agent-scratchpad")
provider = 'AzureChatOpenAI'
model_or_deployment_name = os.getenv('model_name') if provider == 'ChatOpenAI' else os.getenv('deployment_name')
chat_model_1 = get_OpenAI_chain_1(provider, model_or_deployment_name)
parser = ReActOutputJsonKeyStrParser(key='observations')

chain_1 = prompt_1 | chat_model_1 | parser
observations = chain_1.invoke({'task':task, 'scratchpad':scratchpad})
print(observations)

1.  The oldest member of BTS is Jin, and his birth month is December. He was born on December 4, 1992.
2.  The current President of South Korea, Yoon Suk Yeol, was born on 18 December 1960. So, his birth month is December.


### 2. Fetch the prompt and create a chain (using LCEL)

In [18]:
def get_OpenAI_chain_2(provider: Literal['ChatOpenAI', 'AzureChatOpenAI'], model_or_deployment_name: str):
    if provider == 'ChatOpenAI':
        chat_model = ChatOpenAI(model_name=model_or_deployment_name, temperature=0)
    else:
        chat_model = AzureChatOpenAI(deployment_name=model_or_deployment_name, temperature=0)
    return chat_model


prompt_2 = hub.pull("jet-taekyo-lee/response-based-on-observation")
chat_model_2 = get_OpenAI_chain_2(provider, model_or_deployment_name)
chain_2 = prompt_2 | chat_model_2


They are the same.


### 3. Get a response

In [19]:
response = chain_2.invoke({'query':task, 'context':observations})
print(response.content)

They are the same.
